In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import random

from sklearn.metrics import classification_report, roc_auc_score

from sklearn.ensemble import RandomForestClassifier


import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns=None
pd.options.mode.chained_assignment = None

In [2]:
triggers_actions = pd.read_csv(r'C:\Users\User\triggers\triggers_features.csv')

In [3]:
triggers_actions.head()

,Unnamed: 0,guid,date,trigger,type,action_date,result,time_since_last_action,last_successful_type,triggers_since_last_action
0,0,0187808e-d664-cc1d-403d-b93ac1365ee8,2024-05-20 15:30:28,59,1,NaN,NaN,0.000000,NaN,1
1,1,0187808e-d664-cc1d-403d-b93ac1365ee8,2024-05-23 07:04:59,59,1,NaN,NaN,2.648970,NaN,2
2,2,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:45,12,1,NaN,NaN,0.000000,NaN,1
3,3,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:46,27,1,NaN,NaN,0.000012,NaN,2
4,4,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:46,168,1,NaN,NaN,0.000012,NaN,3


In [4]:
# приступим к обработке пропущенных значений
# факт взаимодействия, тут нам не так важна дата, присвоим значением 1 (если взаимодействие было) и 0 (если не было)
triggers_actions['action_date'] = triggers_actions['action_date'].notna().astype(int)

In [5]:
# результат взаимодействия сейчас успешно 1, неуспешно 0, не было - пропуск 
# заменим : не успешно -1, не было 0, успешно 1
triggers_actions['result'] = triggers_actions['result'].replace(0, -1).fillna(0).astype(int)

In [6]:
triggers_actions['result'].value_counts()

result
 0    36762857
-1      367064
 1       11138
Name: count, dtype: int64

In [7]:
# последний успешный тип взаимодействия, категориальный признак, заменим пропуски на '0'
triggers_actions['last_successful_type'] = triggers_actions['last_successful_type'].fillna(0)


In [8]:
triggers_actions['last_successful_type'] = triggers_actions['last_successful_type'].astype('category')

In [9]:
triggers_actions['trigger'] = triggers_actions['trigger'].astype('category')

In [10]:
triggers_actions.head(10)

,Unnamed: 0,guid,date,trigger,type,action_date,result,time_since_last_action,last_successful_type,triggers_since_last_action
0,0,0187808e-d664-cc1d-403d-b93ac1365ee8,2024-05-20 15:30:28,59,1,0,0,0.000000,0.0,1
1,1,0187808e-d664-cc1d-403d-b93ac1365ee8,2024-05-23 07:04:59,59,1,0,0,2.648970,0.0,2
2,2,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:45,12,1,0,0,0.000000,0.0,1
3,3,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:46,27,1,0,0,0.000012,0.0,2
4,4,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:46,168,1,0,0,0.000012,0.0,3
5,5,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:46,133,1,0,0,0.000012,0.0,4
6,6,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:48,188,1,0,0,0.000035,0.0,5
7,7,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:51,6,1,0,0,0.000069,0.0,6
8,8,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:51,243,1,0,0,0.000069,0.0,7
9,9,0187808e-e508-2486-7385-30069e7c0a7f,2024-05-21 18:45:52,155,1,0,0,0.000081,0.0,8


In [11]:
triggers_actions.to_csv(r'C:\Users\User\triggers\before_logreg.csv',index = False)

###  теперь попробуем оценить влияние на успешный результат

In [ ]:
triggers_actions[~triggers_actions['interaction_prob'].isna()].head(30)

In [18]:
smth = triggers_actions[(triggers_actions['result'] == 1) | (triggers_actions['result'] == 0)]

In [ ]:
triggers_actions.head()

In [8]:
# делим данные на train и test, на одних и тех же данных предскажем отдельно веротяность факта взаимодействия и вероятность успеха взаимодействия 

X = triggers_actions[['trigger', 'time_since_last_action', 'triggers_since_last_action', 'last_successful_type']]
y_interaction = triggers_actions['action_date']  # таргет для взаимодействия

# Разделение данных на train/test с фиксацией индексов
X_train, X_test, y_train_interaction, y_test_interaction = train_test_split(X, y_interaction, test_size=0.2, random_state=42)


In [9]:

# Сохраняем индексы, которые попали в train/test
train_indices = X_train.index
test_indices = X_test.index


In [10]:

# Шаг 2: Используем те же индексы для задачи предсказания успеха
y_success = triggers_actions['result']  # таргет для успеха

# Создаем выборки для успеха, используя сохраненные индексы
X_train_success = X.loc[train_indices]
X_test_success = X.loc[test_indices]


In [11]:
y_train_success = y_success.loc[train_indices]
y_test_success = y_success.loc[test_indices]

### модели 

In [12]:
# модель для взаимодействия 
logreg_interaction = LogisticRegression(class_weight='balanced')
logreg_interaction.fit(X_train, y_train_interaction)
interaction_probs = logreg_interaction.predict_proba(X_test)[:, 1]

In [ ]:

# модель для успеха 
logreg_success = LogisticRegression(class_weight='balanced', multi_class='ovr')
logreg_success.fit(X_train_success, y_train_success)
success_probs = logreg_success.predict_proba(X_test_success)[:, 1]


In [ ]:

# добавляем обе вероятности в датафрейм для анализа
triggers_actions.loc[test_indices, 'interaction_prob'] = interaction_probs
triggers_actions.loc[test_indices, 'success_prob'] = success_probs

### посмотрим как связаны вероятности и положительные результат взаимодействия

In [13]:
# возьмём только тестовый набор данных 
filtered_data = triggers_actions[triggers_actions['interaction_prob'].notna()]

In [ ]:

# рисуем гистограмму распределения вероятностей
plt.figure(figsize=(10, 6))
sns.histplot(filtered_data['interaction_prob'], bins=30, kde=True, color='blue')

plt.title('Распределение вероятностей взаимодействия', fontsize=14)
plt.xlabel('Вероятность взаимодействия', fontsize=12)
plt.ylabel('Количество', fontsize=12)

plt.show()

In [ ]:

# рисуем гистограмму распределения вероятностей
plt.figure(figsize=(10, 6))
sns.histplot(filtered_data['success_prob'], bins=30, kde=True, color='blue')

plt.title('Распределение вероятностей успешных взаимодействия', fontsize=14)
plt.xlabel('Вероятность успешного взаимодействия', fontsize=12)
plt.ylabel('Количество', fontsize=12)

plt.show()

In [ ]:

# считаем максимальное значение вероятностей
max_prob = filtered_data['interaction_prob'].max()

# создаем интервалы на основе максимального значения
bins = np.linspace(0, max_prob, 11) 
filtered_data['probability_category'] = pd.cut(filtered_data['interaction_prob'], bins=bins, include_lowest=True)

# считаем количество успешных и неуспешных взаимодействий по категориям
summary = filtered_data.groupby('probability_category')['result'].value_counts().unstack().fillna(0)

# рассчитываем долю успешных взаимодействий
summary['success_rate'] = summary[1] / (summary[1] + summary[0] + summary[-1])

In [ ]:
# строим график success_rate по категориям вероятностей
plt.figure(figsize=(10, 6))
sns.lineplot(x=summary.index.astype(str), y=summary['success_rate'], marker='o', color='green')

# настройки графика
plt.title('Динамика success_rate в зависимости от вероятности взаимодействия', fontsize=14)
plt.xlabel('Категории вероятностей', fontsize=12)
plt.ylabel('Доля успешных взаимодействий (success_rate)', fontsize=12)
plt.xticks(rotation=45)

plt.show()

In [ ]:
filtered_data.head()

In [27]:
# посчитаем P(success) как произведение двух вероятностей, это будет вероятность успеха с учётом вероятности взаимодействовать или нет 
filtered_data['P_success'] = filtered_data['interaction_prob'] * filtered_data['success_prob']


In [ ]:

# Корреляционная матрица для анализа зависимости вероятностей
correlation_matrix = filtered_data[['interaction_prob', 'success_prob', 'P_success']].corr()
print(correlation_matrix)


In [ ]:


# Распределение вероятностей взаимодействия и успеха
plt.figure(figsize=(10, 6))
sns.scatterplot(x='interaction_prob', y='P_success', data=filtered_data, alpha=0.5)
plt.title('Связь вероятности взаимодействия и успеха')
plt.xlabel('Вероятность взаимодействия (P(interaction))')
plt.ylabel('Вероятность успеха (P(success))')
plt.show()


In [ ]:

# Визуализация P(result = 1 | interaction) и P(success)
plt.figure(figsize=(10, 6))
sns.scatterplot(x='success_prob', y='P_success', data=filtered_data, alpha=0.5)
plt.title('Связь P(result = 1 | interaction) и P(success)')
plt.xlabel('Условная вероятность успеха (P(result = 1 | interaction))')
plt.ylabel('Вероятность успеха (P(success))')
plt.show()


In [ ]:

# Шаг 3: Анализ успешности

# Создание категорий для анализа вероятностей
triggers_actions['interaction_category'] = pd.cut(triggers_actions['interaction_prob'], bins=5)
triggers_actions['success_category'] = pd.cut(triggers_actions['success_prob'], bins=5)

# Сводная таблица для анализа успешности по категориям
success_analysis = triggers_actions.groupby(['interaction_category', 'success_category']).agg({
    'P_success': 'mean',  # Средняя вероятность успеха
    'result': lambda x: (x == 1).mean()  # Доля успешных взаимодействий (где результат == 1)
}).reset_index()

print(success_analysis)

### reinforcment learning 

### создание дискретных категорий 

In [25]:
# ВРЕМЯ С ПОСЛДНЕГО ВЗАИМОДЕЙСТВИЯ
min_time = filtered_data['time_since_last_action'].min()
max_time = filtered_data['time_since_last_action'].max()


time_bins = np.linspace(min_time, max_time, 31)

time_bins = sorted(set(time_bins))


filtered_data['time_category'] = pd.cut(filtered_data['time_since_last_action'], bins=time_bins, include_lowest=True)

In [ ]:
filtered_data['time_category'].value_counts()

In [ ]:

# ТРИГЕРЫ
threshold = 3000

max_trigger_value = filtered_data['triggers_since_last_action'].max()
bins = np.linspace(0, threshold, num=30).tolist() 

bins.append(max_trigger_value)  

bins = sorted(set(bins))


filtered_data['triggers_category'] = pd.cut(filtered_data['triggers_since_last_action'], bins=bins, include_lowest=True)
print(filtered_data['triggers_category'].value_counts())


In [ ]:
filtered_data.head()

### создаем q таблиц


In [29]:

# оставляем только необходимые столбцы
filtered_data_time = filtered_data[['guid', 'time_category', 'result']].copy()
# уникальные значения для time_category
time_categories = filtered_data_time['time_category'].unique()

# Подготовка данных для triggers_category
filtered_data_triggers = filtered_data[['guid','result', 'triggers_category']].dropna()

# уникальные значения для triggers_category
triggers_categories = filtered_data_triggers['triggers_category'].unique()

In [ ]:

# Инициализация Q-таблиц
q_table_triggers = pd.DataFrame(index=triggers_categories, columns=[0, 1])
q_table_triggers.fillna(0, inplace=True)

q_table_time = pd.DataFrame(index=time_categories, columns=[0, 1])  # аналогично для времени
q_table_time.fillna(0, inplace=True)

### обучение модели на основе q таблиц 

In [31]:
class QLearningAgent:
    def __init__(self, q_table_triggers, q_table_time, alpha=0.1, gamma=0.9, epsilon=0.9, epsilon_decay=0.99, min_epsilon=0.1):
        self.q_table_triggers = q_table_triggers  # Q-таблица для триггеров
        self.q_table_time = q_table_time  # Q-таблица для времени
        self.alpha = alpha  # скорость обучения
        self.gamma = gamma  # коэффициент дисконтирования
        self.epsilon = epsilon  # параметр ε-жадной стратегии
        self.epsilon_decay = epsilon_decay  # уменьшение ε
        self.min_epsilon = min_epsilon  # минимальное значение для ε

    # выбор действия на основе ε-жадной стратегии
    def choose_action(self, trigger_state, time_state):
        if random.uniform(0, 1) < self.epsilon:  # случайное действие (исследование)
            return random.choice([0, 1])  # случайное действие
        else:  # действие с максимальным Q-значением (эксплуатация)
            # комбинируем значения Q из обеих таблиц
            trigger_q_values = self.q_table_triggers.loc[trigger_state]
            time_q_values = self.q_table_time.loc[time_state]
            combined_q_values = trigger_q_values + time_q_values  # можно использовать сложение или другой метод
            return combined_q_values.idxmax()  # выбираем действие с наибольшим комбинированным Q-значением

    # обновление Q-таблиц
    def update_q_tables(self, trigger_state, time_state, action, reward, next_trigger_state, next_time_state):
        # обновляем Q-таблицу для триггеров
        old_trigger_value = self.q_table_triggers.loc[trigger_state, action]
        next_trigger_max = self.q_table_triggers.loc[next_trigger_state].max()
        new_trigger_value = (1 - self.alpha) * old_trigger_value + self.alpha * (reward + self.gamma * next_trigger_max)
        self.q_table_triggers.loc[trigger_state, action] = new_trigger_value

        # обновляем Q-таблицу для времени
        old_time_value = self.q_table_time.loc[time_state, action]
        next_time_max = self.q_table_time.loc[next_time_state].max()
        new_time_value = (1 - self.alpha) * old_time_value + self.alpha * (reward + self.gamma * next_time_max)
        self.q_table_time.loc[time_state, action] = new_time_value

    # обновление ε для уменьшения вероятности случайных действий
    def decay_epsilon(self):
        self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)


In [ ]:

# Создаем агента
agent = QLearningAgent(q_table_triggers=q_table_triggers, q_table_time=q_table_time)

# Обучение агента
num_episodes = 1000  # количество эпизодов
for episode in range(num_episodes):
    # Случайное состояние (категория триггера и времени)
    trigger_state = random.choice(triggers_categories)
    time_state = random.choice(time_categories)

    # Выбор действия (0 или 1)
    action = agent.choose_action(trigger_state, time_state)

    # Получаем награду за выбранное действие
    reward = filtered_data_triggers[
        (filtered_data_triggers['triggers_category'] == trigger_state) &
        (filtered_data_triggers['result'] == action)
    ]['result'].sum()

    # Случайное следующее состояние
    next_trigger_state = random.choice(triggers_categories)
    next_time_state = random.choice(time_categories)

    # Обновляем Q-таблицы
    agent.update_q_tables(trigger_state, time_state, action, reward, next_trigger_state, next_time_state)

    # Уменьшаем ε (для уменьшения случайных действий)
    agent.decay_epsilon()

# Выводим обновленные Q-таблицы
print("Обученная Q-таблица для триггеров:")
print(agent.q_table_triggers)

In [ ]:
print("Обученная Q-таблица для времени:")
print(agent.q_table_time)